# Review Scraper (working on 26/03/2022) 

**Note**: The scirpt has been written on google colab, so it could give some problems with indendation if you open it with jupyter notebook

In [2]:
import requests
import pprint
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import random 
import re 
import time 

## Description
This dataset contains 1350 reviews scraped from IMDB using BeautifulSoup. Here are listed the columns of the DataFrame and their meaning:
- *Season*: The season to which the review refers;
- *Episode*: The specific episode to which the review refer;
- *Rating*: The score given by the reviewer to the specific episode on a scale to 10;
- *Name*: The name of the reviewer;
- *Date*: The date in which the review has been published;
- *Text*: The text of the review;
- *Useful_rew*: The number of IMDB user that found the specific review useful over the sum of the number of reviewers that find that review useful and the ones who did not. 
- *Useful_score*: Useful_rew expressed as a percentage

### The reviews scraper

In [7]:
reviews_table = pd.DataFrame(columns=['season','episode','Rating', 'Name', 'Date', 'Text', 'Useful']) 
seasons = ['https://www.imdb.com/title/tt0386676/episodes?season='+str(i) for i in range(1,10) ]

for i in range(len(seasons)):
    print('season'+str(i+1))
    reference = []
    hrefs = []
    time.sleep(random.randint(3,7))
    page = requests.get(seasons[i]) 
    soup = BeautifulSoup(page.content, "html.parser") 

    episodes_even = soup.find_all('div', class_='list_item even')
    episodes_odd = soup.find_all('div', class_='list_item odd')

    for ep_e in episodes_even:
        reference.append(ep_e.find('a').find('div').find('div').text)
        hrefs.append(str('https://www.imdb.com/'+ep_e.find('a').get('href')+'reviews'))

    for ep_o in episodes_odd:
        reference.append(ep_o.find('a').find('div').find('div').text)
        hrefs.append(str('https://www.imdb.com/'+ep_o.find('a').get('href')+'reviews'))

#for each episode 
    for ep in range(len(hrefs)):
        season = int(re.findall('[0-9]+',reference[ep])[0])
        episode = int(re.findall('[0-9]+',reference[ep])[1])
        time.sleep(random.randint(3,7))
        page = requests.get(hrefs[ep])
        soup = BeautifulSoup(page.content, "html.parser")

        rating = []         # OK
        reviewer_name = []  # OK
        review_date = []    # OK
        review_text = []    # OK
        useful_number = []  # OK 
        useful_percentage = []

        reviews = soup.find_all('div', class_ = 'lister-item-content')
        season_list = [season for i in reviews] 
        ep_list = [episode for i in reviews] 

        for review in reviews:
            if review.find('span', class_ = 'rating-other-user-rating') == None:
                rating.append('NA')
            else: 
                rating.append(review.find('span', class_ = 'rating-other-user-rating').find('span').text)
      #print(rating)
            reviewer_name.append(review.find('span', class_ = 'display-name-link').text)
            review_date.append(review.find('span', class_ = 'review-date').text)
            review_text.append(review.find('div', class_ = 'content').find('div', class_ = 'text show-more__control').text)
            k = review.find('div', class_ = 'content').find('div', class_ = 'actions text-muted').text
            useful_number.append(str(int(re.findall('[0-9]+',k)[0]))+'/'+str(int(re.findall('[0-9]+',k)[1])))
      
            if int(re.findall('[0-9]+',k)[1]) == 0:
                useful_percentage.append(0)
            else:
                useful_percentage.append(round(int(re.findall('[0-9]+',k)[0])/int(re.findall('[0-9]+',k)[1])*100,0))
      
    

            #print(review.find('div', class_ = 'content').find('div', class_ = 'text show-more__control').text)
        headVarName = ['season','episode','Rating', 'Name', 'Date', 'Text', 'Useful_rew','Useful_score']
        zipped = zip(season_list,ep_list,rating,reviewer_name,review_date,review_text,useful_number,useful_percentage)
        table = pd.DataFrame(zipped, columns = headVarName)
        reviews_table = reviews_table.append(table)


season1
season2
season3
season4
season5
season6
season7
season8
season9


In [8]:
reviews_table

,season,episode,Rating,Name,Date,Text,Useful,Useful_rew,Useful_score
0,1,2,9,ComedyFan2010,27 December 2018,Hilarious episode. After Michael tells some of...,NaN,7/9,78.0
1,1,2,5,studioAT,17 October 2021,This episode just about manages to combine the...,NaN,1/7,14.0
2,1,2,8,Edvis-1997,24 October 2021,"Well, that was so relatable even in Nowadays, ...",NaN,3/4,75.0
3,1,2,9,MaxBorg89,11 February 2009,After following the British template almost sl...,NaN,37/41,90.0
4,1,2,7,zhyarTheChosen,23 February 2020,People actually love this one over the first e...,NaN,1/21,5.0
...,...,...,...,...,...,...,...,...,...
20,9,23,5,HunterGee,11 December 2021,Okay so it sucks when a show loses its main an...,NaN,4/8,50.0
21,9,23,10,naregian,23 May 2013,We all know the show sort of jumped the shark ...,NaN,46/51,90.0
22,9,23,1,Misha0809,18 February 2022,"So, what was the point of bringing Steve Carel...",NaN,0/8,0.0
23,9,23,10,saitanooj-87469,7 April 2020,This show is a stress relief for me particular...,NaN,3/3,100.0


In [ ]:
reviews = reviews_table.sort_values(by = ['season','episode']).reset_index(drop=True)
reviews.to_csv('Reviews.csv',index = False)